In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, avg, col, sum as spark_sum, row_number
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("UserDataAnalysis").getOrCreate()
# 2.1
transaction_df = spark.read.csv("user_balance_table.csv", header=True, inferSchema=True)
user_info_df = spark.read.csv("user_profile_table.csv", header=True, inferSchema=True)
data_df = transaction_df.join(user_info_df, on='user_id')
data_df = data_df.withColumn("report_date", to_date(col("report_date").cast("string"), "yyyyMMdd"))
march1_data = data_df.filter(col("report_date") == '2014-03-01')
avg_balance_per_city = march1_data.groupBy("city").agg(avg("tBalance").alias("avg_tBalance")).orderBy(col("avg_tBalance").desc())
avg_balance_per_city.show()
avg_balance_per_city.write.csv("task2_result/2.1")

# 2.2
august_data = data_df.filter((col("report_date") >= '2014-08-01') & (col("report_date") <= '2014-08-31'))
total_flow_per_user_city = august_data.groupBy("city", "user_id").agg(spark_sum("total_purchase_amt").alias("total_purchase_amt"),spark_sum("total_redeem_amt").alias("total_redeem_amt")).withColumn("total_flow", col("total_purchase_amt") + col("total_redeem_amt"))
window_spec = Window.partitionBy("city").orderBy(col("total_flow").desc())
ranked_users = total_flow_per_user_city.withColumn("rank", row_number().over(window_spec))
top3_users_per_city = ranked_users.filter(col("rank") <= 3).select("city", "user_id", "total_flow", "rank").orderBy("city", "rank")
top3_users_per_city.show()
top3_users_per_city.write.csv("task2_result/2.2")
spark.stop()

+-------+------------------+
|   city|      avg_tBalance|
+-------+------------------+
|6281949| 2795923.837298216|
|6301949|2650775.0664451825|
|6081949|2643912.7566638007|
|6481949|2087617.2136986302|
|6411949|1929838.5617977527|
|6412149| 1896363.471625767|
|6581949|1526555.5551020408|
+-------+------------------+



+-------+-------+----------+----+
|   city|user_id|total_flow|rank|
+-------+-------+----------+----+
|6081949|  27235| 108475680|   1|
|6081949|  27746|  76065458|   2|
|6081949|  18945|  55304049|   3|
|6281949|  15118| 149311909|   1|
|6281949|  11397| 124293438|   2|
|6281949|  25814| 104428054|   3|
|6301949|   2429| 109171121|   1|
|6301949|  26825|  95374030|   2|
|6301949|  10932|  74016744|   3|
|6411949|    662|  75162566|   1|
|6411949|  21030|  49933641|   2|
|6411949|  16769|  49383506|   3|
|6412149|  22585| 200516731|   1|
|6412149|  14472| 138262790|   2|
|6412149|  25147|  70594902|   3|
|6481949|  12026|  51161825|   1|
|6481949|    670|  49626204|   2|
|6481949|  14877|  34488733|   3|
|6581949|   9494|  38854436|   1|
|6581949|  26876|  23449539|   2|
+-------+-------+----------+----+
only showing top 20 rows

